In [8]:

from sympy.combinatorics.graycode import GrayCode
from sympy.combinatorics.graycode import gray_to_bin
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from deap import creator
from deap import base
from deap import tools
from deap import algorithms
from deap import benchmarks
import torch
import random
import array
import numpy as np
import csv

# Load Dataset
batch_size = 10000

transform   = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Function to transform the dataset to the specified range

trainset      = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader   = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset       = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader    = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes       = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Device, ", device)

totalImages   = len(trainloader.dataset)
miniAmount    = 1000
numOfLoaders  = totalImages // miniAmount

print("Number of loaders to create: " + str(numOfLoaders))

miniLoaders = []

for i in range(numOfLoaders):
  startIdx        = i * miniAmount
  endIdx          = (i + 1) * miniAmount if i < numOfLoaders - 1 else totalImages
  subset          = torch.utils.data.Subset(trainset, range(startIdx, endIdx))
  SubTrainLoader  = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=2)

  miniLoaders.append(SubTrainLoader)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 52),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(52, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model         = Net()
PATH          = ('./40_epoch_32_batch_SGD_net.pth')
model.load_state_dict(torch.load(PATH))

finalLayer    = model.classifier[-1]
outerShape    = len(finalLayer.weight)
innerShape    = len(finalLayer.weight[0])
totalWeights  = outerShape * innerShape
biasCount     = len(finalLayer.bias)
paramCount    = sum(param.numel() for param in finalLayer.parameters())

criterion     = nn.CrossEntropyLoss()
nn.init.xavier_uniform(finalLayer.weight)
print("Final layer weight shape,", (finalLayer.weight).shape)
print("Final layer bias shape,", (finalLayer.bias).shape)

for param in model.parameters():
    param.requires_grad = False


# Define parameters

POPULATION_SIZE = 100
LOWER_BOUND     = -1
UPPER_BOUND     = 1
DIMENSION       = 530
dspInterval     = 1
iterations      = 100
crossProb       = 0.8
mutateprob      = .1
nElitists       = 1

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax)

def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

def assign_weights(individual):
    particleweightsNP1 = np.array(individual)
    particleweightsNP = particleweightsNP1[:totalWeights]
    biases = np.array(particleweightsNP1[-biasCount:])
    biases = torch.from_numpy(biases).float()
    finalLayer.bias = torch.nn.Parameter(biases.float())
    reshapedWeights = particleweightsNP.reshape(outerShape,innerShape)
    torchWeights = torch.from_numpy(reshapedWeights).float()
    finalLayer.weight = torch.nn.Parameter(torchWeights.float())

def calcFitness(individual, miniNumber):
    assign_weights(individual)
    correct_train = 0
    total_train = 0

    for data in miniLoaders[miniNumber]:
        model.to(device)
        images, labels = data[0].to(device), data[1].to(device)
        predicted=model(images)

        _, predictions = torch.max(predicted, 1)
        total_train += labels.size(0)
        correct_train += (predictions == labels).sum().item()
    acc = 100*(correct_train/total_train)
    return acc,


toolbox = base.Toolbox()
toolbox.register("attr_float", uniform, LOWER_BOUND, UPPER_BOUND, DIMENSION)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=LOWER_BOUND, up=UPPER_BOUND, eta=20.0)
toolbox.register("mutate",tools.mutPolynomialBounded, low=LOWER_BOUND, up=UPPER_BOUND, eta=20.0,  indpb= 1/DIMENSION)

# set Roulette Wheel selection  for enviromental selection
toolbox.register("select",tools.selRoulette, fit_attr='fitness')
toolbox.register("evaluate", calcFitness)

generations = []
evaluations = []
min_fitness = []
max_fitness = []
avg_fitness = []
std_dev = []

def main():
    #random.seed(64)

    # create an initial population of individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=POPULATION_SIZE)
    miniCounter = 0

    # evaluate the entire population
    fitnesses = list(map(lambda ind: toolbox.evaluate(ind, miniCounter), pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while g < iterations:
        g = g + 1
        print("-- Generation %i --" % g)

        # select the next generation individuals
        offspring = tools.selBest(pop, nElitists) + toolbox.select(pop,len(pop)-nElitists)
        offspring = list(map(toolbox.clone, offspring))

        # apply crossover and mutation on the offspring
        # make pairs of offspring for crossing over
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < crossProb:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability mutateprob
            if random.random() < mutateprob:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(lambda ind: toolbox.evaluate(ind, miniCounter), invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # replace pop with offspring
        pop[:] = offspring

        if g%dspInterval ==0:
            # gather all the fitnesses in one list and print the stats
            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            # append data to lists
            generations.append(g)
            evaluations.append(len(pop))
            min_fitness.append(min(fits))
            max_fitness.append(max(fits))
            avg_fitness.append(mean)
            std_dev.append(std)

            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    miniCounter += 1

    if miniCounter >= (len(miniLoaders)):
        miniCounter = 0
        print("Mini Counter reset!")

if __name__ == "__main__":
    main()


Files already downloaded and verified
Files already downloaded and verified
Device,  cuda:0
Number of loaders to create: 50
Final layer weight shape, torch.Size([10, 52])
Final layer bias shape, torch.Size([10])
Evaluate fitnesses before generation loop..


/tmp/ipykernel_1994428/3207354317.py:111: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(finalLayer.weight)


gen	evals	min  	max  
0  	100  	[4.8]	[17.]
-- Generation 1 --
1  	100  	[3.6]	[19.3]
  Min 3.5999999999999996
  Max 19.3
  Avg 10.588999999999999
  Std 2.994625018261887
-- Generation 2 --
2  	100  	[4.5]	[19.7]
  Min 4.5
  Max 19.7
  Avg 10.944
  Std 2.977291386478645
-- Generation 3 --
3  	100  	[3.7]	[22.7]
  Min 3.6999999999999997
  Max 22.7
  Avg 11.228000000000002
  Std 3.0920245794624543
-- Generation 4 --
4  	100  	[3.9]	[26.8]
  Min 3.9
  Max 26.8
  Avg 11.889000000000003
  Std 3.606186212607434
-- Generation 5 --
5  	100  	[4.5]	[28.] 
  Min 4.5
  Max 28.000000000000004
  Avg 12.658000000000001
  Std 3.706350765915173
-- Generation 6 --
6  	100  	[5.7]	[26.4]
  Min 5.7
  Max 26.400000000000002
  Avg 13.671000000000008
  Std 4.376123741394864
-- Generation 7 --
7  	100  	[6.9]	[25.9]
  Min 6.9
  Max 25.900000000000002
  Avg 14.998000000000008
  Std 4.336057656443197
-- Generation 8 --
8  	100  	[7.] 	[26.1]
  Min 7.000000000000001
  Max 26.1
  Avg 15.957999999999995
  Std 4.0

KeyboardInterrupt: 

In [ ]:
# write data to CSV file
with open('evolution_stats.csv', 'w', newline='') as csvfile:
    fieldnames = ['Generation', 'Evaluations', 'Min Fitness', 'Max Fitness', 'Avg Fitness', 'Std Dev']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(generations)):
        writer.writerow({
            'Generation': generations[i],
            'Evaluations': evaluations[i],
            'Min Fitness': min_fitness[i],
            'Max Fitness': max_fitness[i],
            'Avg Fitness': avg_fitness[i],
            'Std Dev': std_dev[i]
        })